### previously I had problem with index mismatch, after Removing outliers
### I am trying to fix it and get better results as final.

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
df = pd.read_csv('./files_for_lab/csv_files/marketing_customer_analysis.csv')

# Making a copy of the dataset for manipulation
df1 = df.copy()

# Removing outliers from specific columns
def remove_outliers(df, threshold=1.5, in_columns=None, skip_columns=[]):
    if in_columns is None:
        in_columns = df.select_dtypes(np.number).columns
    for column in in_columns:
        if column not in skip_columns:
            upper = np.percentile(df[column], 75)
            lower = np.percentile(df[column], 25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    return df

reset dataframe 1 indexes so that X y will have same index values.
df1.reset_index(drop = True, inplace = True)

In [10]:
# Applying the remove_outliers function
df1 = remove_outliers(df1, threshold=1.5, in_columns=['Total Claim Amount', 'Customer Lifetime Value'])
df1.reset_index(drop=True, inplace=True)

# X-y split
X = df1.drop(["Total Claim Amount"], axis=1)
y = df1["Total Claim Amount"]

# Removing unnecessary columns
X = X.drop(['Customer', 'Effective To Date'], axis=1)

# Normalizing numerical variables
scaler = StandardScaler()
X_num = X.select_dtypes(np.number)
scaler.fit(X_num)
X_num_scaled = scaler.transform(X_num)

# Converting numpy array to Pandas dataframe
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)

# One-hot encoding for categorical variables
X_cat = X.select_dtypes(include=['object'])
X_cat = pd.get_dummies(X_cat)

# Concatenating the numerical and categorical dataframes
X_new = pd.concat([X_num_scaled, X_cat], axis=1)


In [11]:
# Linear Regression Model
model = LinearRegression()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Calculating metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

# Printing metrics
print(f'R2: {r2}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


R2: 0.7856216391382385
MSE: 8669.31480364487
RMSE: 93.10915531592406
MAE: 71.4010801754766


### much better results than before

- R2: 0.7856216391382385
- MSE: 8669.31480364487
- RMSE: 93.10915531592406
- MAE: 71.4010801754766